In [ ]:
import os
import s3fs
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import duckdb


# Configure the S3 filesystem
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

bucket_name = 'sorieux'

# List of specific CSV files to process
csv_files = [
    f's3://{bucket_name}/customers.csv',
    f's3://{bucket_name}/order_details.csv',
    f's3://{bucket_name}/orders.csv',
    f's3://{bucket_name}/products.csv'
]

# Convert each specified CSV file to Parquet
for file in csv_files:
    print(f'Processing file: {file}')
    parquet_file_name = file.replace('.csv', '.parquet')
    parquet_file_path = parquet_file_name

   # Read the CSV file into a DataFrame
    with fs.open(file, 'rb') as f:
        df = pd.read_csv(f)
        
    # Convert the DataFrame to an Arrow table
    table = pa.Table.from_pandas(df)
            
    # Write the table as a Parquet file
    with fs.open(parquet_file_path, 'wb') as f:
        pq.write_table(table, f)

    print(f'Converted {file} to {parquet_file_path}')


In [ ]:
customers_parquet = 's3://sorieux/customers.parquet'
orders_parquet = 's3://sorieux/orders.parquet'

customers_csv = 's3://sorieux/customers.csv'
orders_csv = 's3://sorieux/orders.csv'

con = duckdb.connect(database=':memory:')

In [ ]:
customers_parquet = 's3://sorieux/customers.parquet'
orders_parquet = 's3://sorieux/orders.parquet'

con = duckdb.connect(database=':memory:')



In [ ]:
query = f"""
SELECT 
    c.name,
    COUNT(o.order_id) AS number_of_orders
FROM 
    read_parquet('{orders_parquet}') o
JOIN 
    read_parquet('{customers_parquet}') c 
ON 
    o.customer_id = c.customer_id
GROUP BY 
    c.customer_id, c.name
ORDER BY 
    number_of_orders DESC
LIMIT 5
"""

top_customers = con.execute(query).fetchdf()

In [ ]:
import plotly.graph_objects as go
# Create a leaderboard-style bar chart using Plotly
fig = go.Figure()

# Add bars for each customer
fig.add_trace(go.Bar(
    x=top_customers['number_of_orders'],
    y=top_customers['name'],
    orientation='h',
    marker=dict(
        color=top_customers['number_of_orders'],
        colorscale='Blues',
        showscale=False
    )
))

# Customize the layout for a leaderboard style
fig.update_layout(
    title='Top 5 Customers by Number of Orders',
    title_x=0.5,
    xaxis_title='Number of Orders',
    yaxis_title='Customer Name',
    yaxis=dict(
        autorange='reversed'  # Ensure the highest value is at the top
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(
        size=14,
    ),
    margin=dict(l=150, r=50, t=50, b=50)
)

# Add annotations for the leaderboard
for i, row in top_customers.iterrows():
    fig.add_annotation(
        x=row['number_of_orders'],
        y=row['name'],
        text=f"{row['number_of_orders']} orders",
        showarrow=False,
        font=dict(size=12, color='black'),
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='black',
        borderwidth=1
    )

# Show the plot
fig.show()

![Alt Text](magic.gif)